# Script to diagnose the nature of the eigenvalues from the RSW $\eta$ modal decompostion
### twnh Sep, Oct, Nov '21

#### -1. Housekeeping

In [ ]:
using Gridap, GridapGmsh, GridapMakie
#using GLMakie
#GLMakie.inline!(true)    # Makes sure figures appear in notebook inline.
using CairoMakie
using LinearAlgebra
using NonlinearEigenproblems

using Printf
using Glob
using LaTeXStrings

include("RSW_ModelFunctions.jl")     # Function definitions
checkEigenFlag = true ;       # Check if eigensolution satisfies original nonlinear eigen problem
trimThreshold = 1.0e-6 ;       # Threshold to trim small eigenvalues. Check figures to make sure this is OK.

In [ ]:
#domainName = "square_384nodes_uneven.msh"
#domainName = "square_803nodes_uneven.msh"
domainName = "square_145nodes.msh"
#domainName = "square_514nodes.msh"
#domainName = "circle_268nodes.msh"
#domainName = "circle_992nodes.msh"

# The non-dimensionalization sets the longest lengthscale of the non-dimensional domain equal to one.
 U, V, U∞, V∞, Ω, dΩ, dΓ, t_Γ, n_nodes = DefineProblem(domainName)


In [ ]:

# Physical parameters: Avoid integer multiples of these parameters (they can throw off the polynomial eigensolver)
#F = (2.0 .^ range(-1.001, 11.001, length = 7))
#Fs = (2.0 .^ range(-1.001, 12.001, step = 1.0))
Fs = (2.0 .^ range(-1.001, 12.001, step = 0.125/4))
#Fs = (2.0 .^ range( 2.001, 5.001, step = 0.125/8))
 f = 1.0e-4     # s^{-1}
 g = 9.81     # ms^{-2}
 L = 512.0e3    # m 
 Lᵣ = sqrt.(L^2 ./ Fs)
 H = ((f * Lᵣ) .^ 2) ./ g
 timescale = L ./ sqrt.(g .* H)
@printf("Coriolis parameter                 = [%8.2e] s^{-1}\n", f)
@printf("Gravitational acceleration         = [%8.2f] ms^{-2}\n", g)
@printf("Layer thickness                    = [%8.1f]--[%8.1f] m\n", maximum(H), minimum(H))
@printf("Domain lengthscale                 = [%8.2f] km\n", L / 1e3)
@printf("Deformation radius                 = [%8.2f]--[%8.2f] km\n", maximum(Lᵣ) / 1e3, minimum(Lᵣ) / 1e3)
@printf("Characteristic timescale           = [%8.2f]--[%8.2f] days.\n", maximum(timescale) / 86400, minimum(timescale) / 86400)
@printf("\nInverse Froude number            F = [%8.2f]--[%8.2f] (%d of them)\n", minimum(Fs), maximum(Fs), size(Fs,1))
@printf("Non-dimensional deformation rad.   = [%8.2f]--[%8.2f]\n", maximum(1 ./ sqrt.(Fs)), minimum(1 ./ sqrt.(Fs)))
@printf("\nMesh has                           = [%6.0d] nodes.\n", n_nodes)

In [ ]:
# Loop over different F values calculating the eigenmodes
 NF = size(Fs, 1)
 nev = 3 * n_nodes
ωs = Array{ComplexF64}(undef,nev,NF)
ηs = Array{CellField}(undef,nev,NF)
for ff = 1:NF
    F = Fs[ff]
    ω, _, η, _ = solveModesProblem(F)
    ωs[1:size(ω,1), ff] = ω
    ηs[1:size(η,1), ff] = η
end # ff

In [ ]:
# Plot frequencies
fig = Figure(resolution = (1600,1600))
ax1 = Axis(fig[1, 1], title = "Eigenvalues vs F", xlabel = "F", ylabel = "eigenvalues")
lines!(Fs, sqrt.(Fs),linewidth = 4.0)
lines!(Fs, sqrt.(Fs) .* 0.025,linewidth = 4.0)
for ff = 1:NF
    inds = findall(x -> x > 1.0e-8, real(ωs[:,ff]))
    scatter!(fig[1, 1], Fs[ff] .* ones(size(inds,1)), real(ωs[inds,ff]), markersize = 2, color = :black)
end # ff
ax1.xscale = log2
ax1.yscale = log10
xlims!(ax1,minimum(Fs),maximum(Fs))
ylims!(ax1,1e-2,1e3)
#display(fig)
save("output.pdf",fig)

In [ ]:
# Plot modes
#ffs = 1:2:NF  # F numbers (across columns)
ffs = 1:64:NF  # F numbers (across columns)
mms = 1:8     # modes (down rows)

fig2 = Figure(resolution = (1600, 2000), font =:sans)
for (jj,ff) in zip(1:size(ffs,1),ffs)
  for (ii,mm) in zip(1:size(mms,1),mms)
      thisω = round(real(ωs[mm,ff]),digits=1)
      thisF = round(Fs[ff],digits=1)
      thisη = abs(ηs[mm,ff])
      tit_txt1 = L"$\Re ( \eta_{%$(mm)})$ $\omega = %$(thisω)$, $F = %$(thisF)$"
      ax2 = Axis(fig2[ii,jj],xlabel=L"x",ylabel=L"y",title=tit_txt1)        
      ax2.aspect = DataAspect()
      hidedecorations!(ax2)
      plot!(Ω,thisη)
  end # for
end # for
#display(fig2)
save("output_modes.pdf",fig2)

In [ ]:
ff = 12
this_F = round(Fs[ff],digits=1)
global inds = findall(x -> x > 1.0e-12, real(ωs[:,ff]))
ω = ωs[inds,ff]

fig3 = Figure(resolution = (1200,600))
Axis(fig3[1, 1], title = "Eigenvalue F = $(this_F)", ylabel = "eigenvalues", xlabel = "eigenvalue index")
tmp = abs.(real(ω))
scatter!(fig3[1, 1], tmp, markersize = 4, color = :black)

Axis(fig3[1, 2], title = "Eigenvalue differences F = $(this_F)", ylabel = "differences", xlabel = "eigenvalue index")
tmp = diff(abs.(real(ω)))
scatter!(fig3[1, 2], tmp, markersize = 4, color = :black)

ratiom1 = (abs.(ω[2:end]) ./ abs.(ω[1])) .- 1.0
Axis(fig3[1, 3], title = "Eigenvalue ratios  F = $(this_F)", ylabel = "ratios", xlabel = "eigenvalue index")
scatter!(fig3[1, 3], real(ratiom1), markersize = 4, color = :black)
ylims!(1, 2000)
#display(fig3)
#ratiom1
save("output3.pdf",fig3)